<a href="https://colab.research.google.com/github/Oomycota1/50projects50days/blob/master/GeoDock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GeoDock**: Fast, Flexible Protein-Protein Docking

Official notebook for [GeoDock](www.biorxiv.org/content/10.1101/2023.06.29.547134v1): Flexible Protein-Protein Docking with a Multi-Track Iterative Transformer.

In [1]:
#@title Hit `Runtime` -> `Run all`, then input docking partner1 and partner2

from google.colab import files

out_name = 'test' #@param {type:"string"}

##@markdown Perform structural refinement with OpenMM
#do_refine = False #@param {type:"boolean"}
# refinement is not yet supported
do_refine = False

upload1 = files.upload()
upload2 = files.upload()
partner1 = list(upload1.keys())[0]
partner2 = list(upload2.keys())[0]

Saving 1csp.pdb to 1csp.pdb


Saving 1LRP.pdb to 1LRP.pdb


In [2]:
#@title Install dependencies (~1 min)
%%time
import sys
import torch

pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install geodock
!pip install torch-geometric
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install py3Dmol

# download model weights
!git clone https://github.com/Graylab/GeoDock.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.0 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 134.7 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.6 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch

In [6]:
#@title Run Prediction
from geodock.GeoDockRunner import GeoDockRunner
torch.cuda.empty_cache()
ckpt_file = "/content/GeoDock/geodock/weights/dips_0.3.ckpt"
geodock = GeoDockRunner(ckpt_file=ckpt_file)
pred = geodock.dock(
    partner1=partner1,
    partner2=partner2,
    out_name=out_name,
    do_refine=do_refine,
    use_openmm=True,
)

NameError: name 'filter_backbone' is not defined

In [4]:
#@title Display 3D structure {run: "auto"}
import py3Dmol

color = "chain" #@param ["chain", "lDDT", "rainbow"]

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(out_name+'.pdb','r').read(),'pdb')
if color == 'chain':
  view.setStyle({'chain': 'A'},{'cartoon': {'color': 'green'}})
  view.setStyle({'chain': 'B'},{'cartoon': {'color': 'blue'}})
elif color == 'lDDT':
  view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
elif color == 'rainbow':
  view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

FileNotFoundError: [Errno 2] No such file or directory: 'test.pdb'

In [ ]:
#@title Download results

#@markdown Download the predicted pdb file. If download fails, results are also accessible from file explorer on the left panel of the notebook.

from google.colab import files
files.download(f"{out_name}.pdb")

# Instructions <a name="Instructions"></a>
**Quick start**
1. Enter the name of your output.
2. Press "Runtime" -> "Run all".
3. Upload docking partner1 and partner2.

**Known issues**
* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the large protein complexes.